# Crew AI

### Install Dependencies

In [ ]:
%pip install crewai langchain_community langchain-groq python-dotenv 'crewai[tools]'==0.30.0rc3



### Imports

In [ ]:
import os

from dotenv import load_dotenv 
load_dotenv()
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
# using openai
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Using Groq
# model_name="gemma-7b-it"
model_name="llama3-70b-8192"
# model_name="llama3-8b-8192"
os.environ["OPENAI_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = model_name
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

# Groq
from langchain_groq import ChatGroq

# CrewAI
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, tool


### Init Groq

In [ ]:
groq_llm = ChatGroq(
            temperature=0, 
            model_name=model_name
        )

### Tools

In [ ]:
search_tool = SerperDevTool()


@tool("Spelling Checker")
def spelling_checker(text: str) -> str:
    """Useful when you finish a article and want to check for spelling mistakes"""
    
    return text + " Checked!!!"


### Create the Agents

In [ ]:
# Creating a senior researcher agent with memory and verbose mode
editor = Agent(
  role='Editor',
  goal='Oversee the creation of a high-quality blog post and tweet ',
  verbose=True,
  backstory=(
    "As an Editor, you are responsible for overseeing the creation of high-quality blog posts and tweets. "
    "You have a keen eye for detail and a deep understanding of the target audience. You ensure that the content is engaging, "
    "informative, and aligns with the brand's voice and style. You work closely with the writers to provide feedback and guidance "
    "to help them produce their best work."
  ),
  allow_delegation=True,
  max_iter=10,
  llm=groq_llm
)
researcher = Agent(
  role='Researcher',
  goal='Research specific topics on the web and provide summarized information',
  verbose=True,
  backstory=(
    "Equipped with web search specialized in extracting and summarizing the most relevant information from a vast array "
    "of online resources. Using skills to stay updated on emerging trends and essential data."
  ),
  tools=[search_tool],
  allow_delegation=True,
  llm=groq_llm
)

# Creating a writer agent with custom tools and delegation capability
writer = Agent(
  role='Writer',
  goal='Create engaging and informative blog posts on assigned topics',
  verbose=True,
  backstory=(
        "As a dedicated Blog Writer within a dynamic digital content team, you excel in transforming "
        "complex topics into readable, engaging blog posts that capture readers' interest and convey information "
        "effectively. Specializing in creating content that is not only informative but also captivates the audience, "
        "using a rich understanding of language and audience engagement without the need for external tools."
    ),
  tools=[search_tool],
  allow_delegation=False,
  llm=groq_llm
)

### Create the Tasks

In [ ]:
# Research task
research_task = Task(
  description=(
    "Identify the trend in {topic}."
    "Your final report should clearly articulate the key points,"
    "its market opportunities, and potential risks."
  ),
  expected_output='A comprehensive report on the latest {topic} trends. Do not use any tools to summarize the information.',
  tools=[search_tool],
  agent=researcher,
  async_execution=False,
  output_file='./output/research-output.md'
)

# Writing task with language model configuration
write_blog = Task(
  description=(
    "Compose an insightful article on {topic}."
    "Focus on the latest trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive."
  ),
  expected_output='A 6 paragraph article on {topic} advancements formatted as markdown.',
  agent=writer,
  tools=[search_tool],
  async_execution=False,
  output_file='./output/blog-post.md',
)

write_tweet = Task(
  description=(
   "Based the provided article about {topic} craft a tweet that captures the article's essence "
   " in a compelling way. Add hashtags and mentions as needed."
  ),
  expected_output='A tweet-length summary of the article.',
  agent=writer,
  tools=[],
  async_execution=False,
  output_file='./output/tweet.md',
)

### Create the Crew

In [ ]:
# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[editor, researcher, writer],
  tasks=[research_task, write_blog, write_tweet],
  process=Process.hierarchical,  # Optional: Sequential task execution is default
  manager_llm=groq_llm,
  cache=True,
  max_rpm=29, 
  verbose=2,
)

### Initiate the work

In [ ]:
result = crew.kickoff(inputs={'topic': 'AI Agents'})
print("--------------------")
print(result)